# Project to understand Linear regression, Classification  and Regression in Python

We will take sklearn's housing dataset available. This dataset contains information about housing values in suburban areas, such as the median value of owner-occupied homes, along with various other attributes like crime rate, proximity to highways, and socioeconomic factors. This dataset is commonly used for regression tasks to predict housing prices based on these features.

**Data source:** [lib.stat.cmu.edu/datasets/boston](lib.stat.cmu.edu/datasets/boston)


 The Boston house-price data of Harrison, D. and Rubinfeld, D.L. 'Hedonic  prices and the demand for clean air', J. Environ. Economics & Management, vol.5, 81-102, 1978.   Used in Belsley, Kuh & Welsch, 'Regression diagnostics
 ...', Wiley, 1980.   N.B. Various transformations are used in the table on pages 244-261 of the latter.

 Variables in order:
- **CRIM:**    per capita crime rate by town
- **ZN:**       proportion of residential land zoned for lots over 25,000 sq.ft.
- **INDUS:**    proportion of non-retail business acres per town
- **CHAS:**     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
- **NOX:**      nitric oxides concentration (parts per 10 million)
- **RM:**       average number of rooms per dwelling
- **AGE:**      proportion of owner-occupied units built prior to 1940
- **DIS:**      weighted distances to five Boston employment centres
- **RAD:**      index of accessibility to radial highways
- **TAX:**      full-value property-tax rate per $10,000
- **PTRATIO:**  pupil-teacher ratio by town
- **B:**        1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
- **LSTAT:**    % lower status of the population
- **MEDV:**     Median value of owner-occupied homes in $1000's


In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import requests

%matplotlib inline

In [2]:
# URL of the dataset
url = "http://lib.stat.cmu.edu/datasets/boston"

# Send a GET request to fetch the data
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Decode the content of the response
    content = response.content.decode("utf-8")
    
    # Save the content to a CSV file
    with open("boston_dataset.csv", "w") as f:
        f.write(content)

    print("Data saved successfully as boston_dataset.csv")

else:
    print("Failed to fetch data from the URL")


Data saved successfully as boston_dataset.csv


In [3]:
import pandas as pd

# Define column names
column_names = [
    "CRIM", "ZN", "INDUS", "CHAS", "NOX", 
    "RM", "AGE", "DIS", "RAD", "TAX", 
    "PTRATIO", "B", "LSTAT", "MEDV"
]

# Read the lines from the file and drop the top 22 rows
with open("boston_dataset.csv", "r") as file:
    lines = file.readlines()[22:]

# Merge every two lines into one and split into columns
data = []
for i in range(0, len(lines), 2):
    row_data = lines[i].strip().split()[:13]  # Take the first 13 columns from the first line
    row_data += lines[i+1].strip().split()   # Take all columns from the second line
    data.append(row_data)

# Create DataFrame
df = pd.DataFrame(data, columns=column_names)

# Display the DataFrame
df.head()


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.00,2.310,0,0.5380,6.5750,65.20,4.0900,1,296.0,15.30,396.90,4.98,24.00
1,0.02731,0.00,7.070,0,0.4690,6.4210,78.90,4.9671,2,242.0,17.80,396.90,9.14,21.60
2,0.02729,0.00,7.070,0,0.4690,7.1850,61.10,4.9671,2,242.0,17.80,392.83,4.03,34.70
3,0.03237,0.00,2.180,0,0.4580,6.9980,45.80,6.0622,3,222.0,18.70,394.63,2.94,33.40
4,0.06905,0.00,2.180,0,0.4580,7.1470,54.20,6.0622,3,222.0,18.70,396.90,5.33,36.20


In [4]:
type(df)

pandas.core.frame.DataFrame

### Dividing the dataset into independent and dependent variables

In [5]:
# Set "MEDV" as the target column
target_column = "MEDV"
X = df.drop(columns=[target_column]) # Independent variable
y = df[target_column] # dependent variable

# Rename the target column to "Price"
y = y.rename("Price")

In [6]:
y

0      24.00
1      21.60
2      34.70
3      33.40
4      36.20
       ...  
501    22.40
502    20.60
503    23.90
504    22.00
505    11.90
Name: Price, Length: 506, dtype: object

In [7]:
X

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.00,2.310,0,0.5380,6.5750,65.20,4.0900,1,296.0,15.30,396.90,4.98
1,0.02731,0.00,7.070,0,0.4690,6.4210,78.90,4.9671,2,242.0,17.80,396.90,9.14
2,0.02729,0.00,7.070,0,0.4690,7.1850,61.10,4.9671,2,242.0,17.80,392.83,4.03
3,0.03237,0.00,2.180,0,0.4580,6.9980,45.80,6.0622,3,222.0,18.70,394.63,2.94
4,0.06905,0.00,2.180,0,0.4580,7.1470,54.20,6.0622,3,222.0,18.70,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.00,11.930,0,0.5730,6.5930,69.10,2.4786,1,273.0,21.00,391.99,9.67
502,0.04527,0.00,11.930,0,0.5730,6.1200,76.70,2.2875,1,273.0,21.00,396.90,9.08
503,0.06076,0.00,11.930,0,0.5730,6.9760,91.00,2.1675,1,273.0,21.00,396.90,5.64
504,0.10959,0.00,11.930,0,0.5730,6.7940,89.30,2.3889,1,273.0,21.00,393.45,6.48


## Linear regression

In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

lin_rag = LinearRegression() 
mse = cross_val_score(lin_rag, X,y,scoring='neg_mean_squared_error', cv=5)
print(mse)

[-12.46030057 -26.04862111 -33.07413798 -80.76237112 -33.31360656]


In [9]:
mean_mse = np.mean(mse)
print(mean_mse)

-37.13180746769923


In [10]:
from sklearn.model_selection import train_test_split
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
fit = lin_rag.fit(X_train, y_train)

In [12]:
# 3. Make predictions using the trained model
predictions = lin_rag.predict(X_test)
predictions

array([28.99672362, 36.02556534, 14.81694405, 25.03197915, 18.76987992,
       23.25442929, 17.66253818, 14.34119   , 23.01320703, 20.63245597,
       24.90850512, 18.63883645, -6.08842184, 21.75834668, 19.23922576,
       26.19319733, 20.64773313,  5.79472718, 40.50033966, 17.61289074,
       27.24909479, 30.06625441, 11.34179277, 24.16077616, 17.86058499,
       15.83609765, 22.78148106, 14.57704449, 22.43626052, 19.19631835,
       22.43383455, 25.21979081, 25.93909562, 17.70162434, 16.76911711,
       16.95125411, 31.23340153, 20.13246729, 23.76579011, 24.6322925 ,
       13.94204955, 32.25576301, 42.67251161, 17.32745046, 27.27618614,
       16.99310991, 14.07009109, 25.90341861, 20.29485982, 29.95339638,
       21.28860173, 34.34451856, 16.04739105, 26.22562412, 39.53939798,
       22.57950697, 18.84531367, 32.72531661, 25.0673037 , 12.88628956,
       22.68221908, 30.48287757, 31.52626806, 15.90148607, 20.22094826,
       16.71089812, 20.52384893, 25.96356264, 30.61607978, 11.59

### [Ridge Lasso regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html)

`class sklearn.linear_model.Ridge(alpha=1.0, *, fit_intercept=True, copy_X=True, max_iter=None, tol=0.0001, solver='auto', positive=False, random_state=None)`

- alpha{float, ndarray of shape (n_targets,)}, default=1.0 : Constant that multiplies the L2 term, controlling regularization strength. alpha must be a non-negative float i.e. in [0, inf).

In [13]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

ridge =Ridge()
params = {'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,20]}
ridge_regressor = GridSearchCV(ridge, params, scoring='neg_mean_squared_error', cv=5)
ridge_regressor.fit(X,y)

GridSearchCV(cv=5, estimator=Ridge(),
             param_grid={'alpha': [1e-15, 1e-10, 1e-08, 0.001, 0.01, 1, 5, 10,
                                   20]},
             scoring='neg_mean_squared_error')

In [14]:
print(ridge_regressor.best_params_)
print(ridge_regressor.best_score_)

{'alpha': 20}
-32.38025025182519


Now MSE is reduced from -37 to -32

- From Linear regression = -37 
- From Ridge regression = -32

### Lasso regression

In [15]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

lasso =Lasso()
params = {'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,20]}
lasso_regressor = GridSearchCV(lasso, params, scoring='neg_mean_squared_error', cv=5)
lasso_regressor.fit(X,y)

c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.633e+03, tolerance: 3.919e+00
  model = cd_fast.enet_coordinate_descent(
c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.760e+03, tolerance: 3.307e+00
  model = cd_fast.enet_coordinate_descent(
c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the

GridSearchCV(cv=5, estimator=Lasso(),
             param_grid={'alpha': [1e-15, 1e-10, 1e-08, 0.001, 0.01, 1, 5, 10,
                                   20]},
             scoring='neg_mean_squared_error')

In [16]:
print(lasso_regressor.best_params_)
print(lasso_regressor.best_score_)

{'alpha': 1}
-35.53158022069486


**Now putting more alpha values**

In [19]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

ridge1 =Ridge()
params = {'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,20,35,40,50,55,60,80,90,100]}
ridge_regressor1 = GridSearchCV(ridge1, params, scoring='neg_mean_squared_error', cv=5)
ridge_regressor1.fit(X,y)

print(ridge_regressor1.best_params_)
print(ridge_regressor1.best_score_)

{'alpha': 100}
-29.905701947540354


In [20]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

lasso1 =Lasso()
params = {'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,20,35,40,50,55,60,80,90,100]}
lasso_regressor1 = GridSearchCV(lasso1, params, scoring='neg_mean_squared_error', cv=5)
lasso_regressor1.fit(X,y)

c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.633e+03, tolerance: 3.919e+00
  model = cd_fast.enet_coordinate_descent(
c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.760e+03, tolerance: 3.307e+00
  model = cd_fast.enet_coordinate_descent(
c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the

GridSearchCV(cv=5, estimator=Lasso(),
             param_grid={'alpha': [1e-15, 1e-10, 1e-08, 0.001, 0.01, 1, 5, 10,
                                   20, 35, 40, 50, 55, 60, 80, 90, 100]},
             scoring='neg_mean_squared_error')

In [21]:
print(lasso_regressor.best_params_)
print(lasso_regressor.best_score_)

{'alpha': 1}
-35.53158022069486


In [23]:
print(f'Ridge for less number of alpha {ridge_regressor.best_score_}')
print(f'Ridge when large number of alpha {ridge_regressor1.best_score_}')
print(f'Lasso for less number of alpha {lasso_regressor.best_score_}')
print(f'Lasso when large number of alpha {lasso_regressor1.best_score_}')

Ridge for less number of alpha -32.38025025182519
Ridge when large number of alpha -29.905701947540354
Lasso for less number of alpha -35.53158022069486
Lasso when large number of alpha -35.53158022069486


**Now let's use train test method:**

In [24]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [26]:
import warnings
from sklearn.exceptions import ConvergenceWarning

# Suppress convergence warnings
warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [28]:
lin_rag2 = LinearRegression() 
mse = cross_val_score(lin_rag, X_train,y_train,scoring='neg_mean_squared_error', cv=5)
mean_mse = np.mean(mse)
print(f'Linear regression = {mean_mse}')

ridge2 =Ridge()
params = {'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,20,35,40,50,55,60,80,90,100]}
ridge_regressor2 = GridSearchCV(ridge2, params, scoring='neg_mean_squared_error', cv=5)
ridge_regressor2.fit(X_train,y_train)

print(f'Ridge regression = {ridge_regressor2.best_params_}')
print(f'Ridge regression = {ridge_regressor2.best_score_}')

lasso2 =Lasso()
params = {'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,20,35,40,50,55,60,80,90,100]}
lasso_regressor2 = GridSearchCV(lasso2, params, scoring='neg_mean_squared_error', cv=5)
lasso_regressor2.fit(X_train,y_train)

print(f'Lasso regression = {lasso_regressor2.best_params_}')
print(f'Lasso regression = {lasso_regressor2.best_score_}')

Linear regression = -25.18787473928514
Ridge regression = {'alpha': 0.01}
Ridge regression = -25.18689936738697
Lasso regression = {'alpha': 1e-15}
Lasso regression = -25.18787473928503


In [30]:
y_pred = lin_rag.predict(X_test)
from sklearn.metrics import r2_score

r2_score_lin = r2_score(y_pred, y_test)

print(r2_score_lin)

0.6832260852266521


In [31]:
y_pred = ridge_regressor2.predict(X_test)
from sklearn.metrics import r2_score

r2_score_ridge = r2_score(y_pred, y_test)

print(r2_score_ridge)

0.6708743257533069


In [32]:
y_pred = lasso_regressor2.predict(X_test)
from sklearn.metrics import r2_score

r2_score_lasso = r2_score(y_pred, y_test)

print(r2_score_lasso)

0.670955897674443


## **Logistic regression**

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

`class sklearn.linear_model.LogisticRegression(penalty='l2', *, dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver='lbfgs', max_iter=100, multi_class='auto', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None)`

In [5]:
from sklearn.linear_model import LogisticRegression 
from sklearn.datasets import load_breast_cancer

df = load_breast_cancer()

# Independent features
X = pd.DataFrame(df['data'], columns=df['feature_names'])

In [6]:
X.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [7]:
# Depednent feature 
y = pd.DataFrame(df['target'], columns=["Target"])

In [8]:
y

,Target
0,0
1,0
2,0
3,0
4,0
...,...
564,0
565,0
566,0
567,0


In [9]:
# lets check the dataset is imbalanced 
y['Target'].value_counts()

Target
1    357
0    212
Name: count, dtype: int64

probably it is a balanced dataset.

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [11]:
params = [{'C': [1,5,10]}, {'max_iter': [100,150]}]

In [12]:
model1 = LogisticRegression(C=100, max_iter=100)

In [ ]:
GridSearchCV(model1, param_grid=params, scoring='f1')